In [1]:
import numpy as np
import pandas as pd
import random    

# ---------------------------
seed_num = 42
# ---------------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4069), (6182,), (1545, 10, 4069), (1545,))

# Adaboost-LSTM
- https://github.com/veniversum/keras/blob/9a401eb2e184fda7238a6259c1b8b02c645e4e9c/keras/wrappers/scikit_learn.py

In [2]:
import sklearn
import sklearn.utils

"""Wrapper for using the Scikit-Learn API with Keras models.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import copy
import types

import sklearn
import keras
from keras.utils.np_utils import to_categorical
from keras.utils.generic_utils import has_arg
from keras.models import Sequential


class BaseWrapper(object):
    """Base class for the Keras scikit-learn wrapper.

    Warning: This class should not be used directly.
    Use descendant classes instead.

    # Arguments
        build_fn: callable function or class instance
        **sk_params: model parameters & fitting parameters

    The `build_fn` should construct, compile and return a Keras model, which
    will then be used to fit/predict. One of the following
    three values could be passed to `build_fn`:
    1. A function
    2. An instance of a class that implements the `__call__` method
    3. None. This means you implement a class that inherits from either
    `KerasClassifier` or `KerasRegressor`. The `__call__` method of the
    present class will then be treated as the default `build_fn`.

    `sk_params` takes both model parameters and fitting parameters. Legal model
    parameters are the arguments of `build_fn`. Note that like all other
    estimators in scikit-learn, `build_fn` should provide default values for
    its arguments, so that you could create the estimator without passing any
    values to `sk_params`.

    `sk_params` could also accept parameters for calling `fit`, `predict`,
    `predict_proba`, and `score` methods (e.g., `epochs`, `batch_size`).
    fitting (predicting) parameters are selected in the following order:

    1. Values passed to the dictionary arguments of
    `fit`, `predict`, `predict_proba`, and `score` methods
    2. Values passed to `sk_params`
    3. The default values of the `keras.models.Sequential`
    `fit`, `predict`, `predict_proba` and `score` methods

    When using scikit-learn's `grid_search` API, legal tunable parameters are
    those you could pass to `sk_params`, including fitting parameters.
    In other words, you could use `grid_search` to search for the best
    `batch_size` or `epochs` as well as the model parameters.
    """

    def __init__(self, build_fn=None, **sk_params):
        self.build_fn = build_fn
        self.sk_params = sk_params
        self.check_params(sk_params)

    def check_params(self, params):
        """Checks for user typos in `params`.

        # Arguments
            params: dictionary; the parameters to be checked

        # Raises
            ValueError: if any member of `params` is not a valid argument.
        """
        legal_params_fns = [Sequential.fit, Sequential.predict,
                            Sequential.predict_classes, Sequential.evaluate]
        if self.build_fn is None:
            legal_params_fns.append(self.__call__)
        elif (not isinstance(self.build_fn, types.FunctionType) and
              not isinstance(self.build_fn, types.MethodType)):
            legal_params_fns.append(self.build_fn.__call__)
        else:
            legal_params_fns.append(self.build_fn)

        for params_name in params:
            for fn in legal_params_fns:
                if has_arg(fn, params_name):
                    break
            else:
                if params_name != 'nb_epoch':
                    raise ValueError(
                        '{} is not a legal parameter'.format(params_name))

    def get_params(self, **params):
        """Gets parameters for this estimator.

        # Arguments
            **params: ignored (exists for API compatibility).

        # Returns
            Dictionary of parameter names mapped to their values.
        """
        res = copy.deepcopy(self.sk_params)
        res.update({'build_fn': self.build_fn})
        return res

    def set_params(self, **params):
        """Sets the parameters of this estimator.

        # Arguments
            **params: Dictionary of parameter names mapped to their values.

        # Returns
            self
        """
        self.check_params(params)
        self.sk_params.update(params)
        return self

    def fit(self, x, y, **kwargs):
        """Constructs a new model with `build_fn` & fit the model to `(x, y)`.

        # Arguments
            x : array-like, shape `(n_samples, n_features)`
                Training samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            y : array-like, shape `(n_samples,)` or `(n_samples, n_outputs)`
                True labels for `x`.
            **kwargs: dictionary arguments
                Legal arguments are the arguments of `Sequential.fit`

        # Returns
            history : object
                details about the training history at each epoch.
        """
        if self.build_fn is None:
            self.model = self.__call__(**self.filter_sk_params(self.__call__))
        elif (not isinstance(self.build_fn, types.FunctionType) and
              not isinstance(self.build_fn, types.MethodType)):
            self.model = self.build_fn(
                **self.filter_sk_params(self.build_fn.__call__))
        else:
            self.model = self.build_fn(**self.filter_sk_params(self.build_fn))

        loss_name = self.model.loss
        if hasattr(loss_name, '__name__'):
            loss_name = loss_name.__name__
        if loss_name == 'categorical_crossentropy' and len(y.shape) != 2:
            y = to_categorical(y)

        fit_args = copy.deepcopy(self.filter_sk_params(Sequential.fit))
        fit_args.update(kwargs)

        history = self.model.fit(x, y, **fit_args)

        return history

    def filter_sk_params(self, fn, override=None):
        """Filters `sk_params` and returns those in `fn`'s arguments.

        # Arguments
            fn : arbitrary function
            override: dictionary, values to override `sk_params`

        # Returns
            res : dictionary containing variables
                in both `sk_params` and `fn`'s arguments.
        """
        override = override or {}
        res = {}
        for name, value in self.sk_params.items():
            if has_arg(fn, name):
                res.update({name: value})
        res.update(override)
        return res


class KerasClassifier(BaseWrapper):
    """Implementation of the scikit-learn classifier API for Keras.
    """

    def fit(self, x, y, sample_weight=None, **kwargs):
        """Constructs a new model with `build_fn` & fit the model to `(x, y)`.

        # Arguments
            x : array-like, shape `(n_samples, n_features)`
                Training samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            y : array-like, shape `(n_samples,)` or `(n_samples, n_outputs)`
                True labels for `x`.
            **kwargs: dictionary arguments
                Legal arguments are the arguments of `Sequential.fit`

        # Returns
            history : object
                details about the training history at each epoch.

        # Raises
            ValueError: In case of invalid shape for `y` argument.
        """
        y = np.array(y)
        if len(y.shape) == 2 and y.shape[1] > 1:
            self.classes_ = np.arange(y.shape[1])
        elif (len(y.shape) == 2 and y.shape[1] == 1) or len(y.shape) == 1:
            self.classes_ = np.unique(y)
            y = np.searchsorted(self.classes_, y)
        else:
            raise ValueError('Invalid shape for y: ' + str(y.shape))
        self.n_classes_ = len(self.classes_)
        if sample_weight is not None:
            kwargs['sample_weight'] = sample_weight
        return super(KerasClassifier, self).fit(x, y, **kwargs)

    def predict(self, x, **kwargs):
        """Returns the class predictions for the given test data.

        # Arguments
            x: array-like, shape `(n_samples, n_features)`
                Test samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            **kwargs: dictionary arguments
                Legal arguments are the arguments
                of `Sequential.predict_classes`.

        # Returns
            preds: array-like, shape `(n_samples,)`
                Class predictions.
        """
        kwargs = self.filter_sk_params(Sequential.predict_classes, kwargs)
        classes = self.model.predict_classes(x, **kwargs)
        return self.classes_[classes]

    def predict_proba(self, x, **kwargs):
        """Returns class probability estimates for the given test data.

        # Arguments
            x: array-like, shape `(n_samples, n_features)`
                Test samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            **kwargs: dictionary arguments
                Legal arguments are the arguments
                of `Sequential.predict_classes`.

        # Returns
            proba: array-like, shape `(n_samples, n_outputs)`
                Class probability estimates.
                In the case of binary classification,
                to match the scikit-learn API,
                will return an array of shape `(n_samples, 2)`
                (instead of `(n_sample, 1)` as in Keras).
        """
        kwargs = self.filter_sk_params(Sequential.predict_proba, kwargs)
        probs = self.model.predict_proba(x, **kwargs)

        # check if binary classification
        if probs.shape[1] == 1:
            # first column is probability of class 0 and second is of class 1
            probs = np.hstack([1 - probs, probs])
        return probs

    def score(self, x, y, **kwargs):
        """Returns the mean accuracy on the given test data and labels.

        # Arguments
            x: array-like, shape `(n_samples, n_features)`
                Test samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            y: array-like, shape `(n_samples,)` or `(n_samples, n_outputs)`
                True labels for `x`.
            **kwargs: dictionary arguments
                Legal arguments are the arguments of `Sequential.evaluate`.

        # Returns
            score: float
                Mean accuracy of predictions on `x` wrt. `y`.

        # Raises
            ValueError: If the underlying model isn't configured to
                compute accuracy. You should pass `metrics=["accuracy"]` to
                the `.compile()` method of the model.
        """
        y = np.searchsorted(self.classes_, y)
        kwargs = self.filter_sk_params(Sequential.evaluate, kwargs)

        loss_name = self.model.loss
        if hasattr(loss_name, '__name__'):
            loss_name = loss_name.__name__
        if loss_name == 'categorical_crossentropy' and len(y.shape) != 2:
            y = to_categorical(y)

        outputs = self.model.evaluate(x, y, **kwargs)
        if not isinstance(outputs, list):
            outputs = [outputs]
        for name, output in zip(self.model.metrics_names, outputs):
            if name == 'acc':
                return output
        raise ValueError('The model is not configured to compute accuracy. '
                         'You should pass `metrics=["accuracy"]` to '
                         'the `model.compile()` method.')

Using TensorFlow backend.


In [3]:
# ---------------------------
seed_num = 42
# ---------------------------

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, LSTM, InputLayer
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier
from sklearn import metrics 
from tensorflow import keras

def get_model(seed_num):
    tf.random.set_seed(seed_num)

    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))
    optimizer = keras.optimizers.Adam(learning_rate = 0.01, decay=0.1)
    lstm.compile(optimizer= optimizer, loss = "binary_crossentropy", metrics=['acc'])
    return lstm

In [4]:
lstm_Predictors = KerasClassifier(build_fn=lambda:get_model(seed_num), epochs=50, batch_size=128)
final_model = AdaBoostClassifier(lstm_Predictors, n_estimators=3, random_state=42, learning_rate=0.01)
final_model.fit(X_train,y_train)

Epoch 1/50
49/49 [==============================] - 8s 69ms/step - loss: 1.0956e-04 - acc: 0.5977
Epoch 2/50
49/49 [==============================] - 3s 65ms/step - loss: 1.0880e-04 - acc: 0.6098
Epoch 3/50
49/49 [==============================] - 3s 65ms/step - loss: 1.0895e-04 - acc: 0.6100
Epoch 4/50
49/49 [==============================] - 3s 65ms/step - loss: 1.0871e-04 - acc: 0.6082
Epoch 5/50
49/49 [==============================] - 3s 64ms/step - loss: 1.0874e-04 - acc: 0.6085
Epoch 6/50
49/49 [==============================] - 3s 65ms/step - loss: 1.0867e-04 - acc: 0.6093
Epoch 7/50
49/49 [==============================] - 3s 64ms/step - loss: 1.0875e-04 - acc: 0.6113
Epoch 8/50
49/49 [==============================] - 3s 65ms/step - loss: 1.0856e-04 - acc: 0.6110
Epoch 9/50
49/49 [==============================] - 3s 63ms/step - loss: 1.0909e-04 - acc: 0.6095
Epoch 10/50
 6/49 [==>...........................] - ETA: 2s - loss: 1.0884e-04 - acc: 0.6029

KeyboardInterrupt: 

In [ ]:
preds = final_model.predict(X_test) # 분류결과 return 
from sklearn import metrics
print('정확도 :', metrics.accuracy_score(y_test, preds))

# Voting-LSTM

In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, LSTM, InputLayer
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.ensemble import VotingClassifier
from sklearn import metrics 
from tqdm import tqdm

# ---------------------------
seed_num = 42
# ---------------------------

tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))
lstm.compile(optimizer= "adam", loss = "binary_crossentropy", metrics=['acc'])

estimator = []
for i in range(1,3):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:lstm, epochs=10, batch_size=256)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator) 
voting_model = VotingClassifier(estimators=estimator, voting = 'soft')


[('model1', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd4f07b8310>), ('model2', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd4f074fb80>)]


In [10]:
voting_model.fit(X_train,y_train)

Epoch 1/10
25/25 [==============================] - 7s 119ms/step - loss: 0.6792 - acc: 0.6000
Epoch 2/10
25/25 [==============================] - 3s 120ms/step - loss: 0.6717 - acc: 0.6106
Epoch 3/10
25/25 [==============================] - 3s 118ms/step - loss: 0.6600 - acc: 0.6136
Epoch 4/10
25/25 [==============================] - 3s 119ms/step - loss: 0.5721 - acc: 0.7048
Epoch 5/10
25/25 [==============================] - 3s 116ms/step - loss: 0.4840 - acc: 0.7800
Epoch 6/10
25/25 [==============================] - 3s 118ms/step - loss: 0.4280 - acc: 0.8120
Epoch 7/10
25/25 [==============================] - 3s 115ms/step - loss: 0.3964 - acc: 0.8310
Epoch 8/10
25/25 [==============================] - 3s 116ms/step - loss: 0.3563 - acc: 0.8598
Epoch 9/10
25/25 [==============================] - 3s 106ms/step - loss: 0.3351 - acc: 0.8701
Epoch 10/10
25/25 [==============================] - 3s 114ms/step - loss: 0.3149 - acc: 0.8816
Epoch 1/10
25/25 [==============================]

VotingClassifier(estimators=[('model1',
                              <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd4f07b8310>),
                             ('model2',
                              <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd4f074fb80>)],
                 voting='soft')

In [11]:
preds = voting_model.predict(X_test)
from sklearn import metrics
print('정확도 :', metrics.accuracy_score(y_test, preds))

정확도 : 0.7527508090614887
